In [4]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [5]:
X, y = make_regression(n_samples=10000, n_features=100, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)

In [6]:
class AdamLinreg:
    def __init__(self, learning_rate:float=0.001, decay_rates:tuple=(0.9, 0.999)) -> None:
        self.learning_rate = learning_rate
        self.decay_rates = decay_rates
        self.epsilon = 10**(-8)
        
        self.m_moment = np.array([], dtype=float)
        self.v_moment = np.array([], dtype=float)
        
        self.losses = np.array([], dtype=float)
    
    def init_weights(self, n_features:int) -> None:
        self.weights = np.zeros(n_features, dtype=float)
        
    def calculate_grads(self, x:list[float], y:float, target:float) -> list[float]:
        return [2 * (y - target) * x[i] for i in range(len(x))]
    
    def calculate_moments(self, grads:list[float], n_iter:int) -> tuple[float]:
        m = self.decay_rates[0] * self.m_moment[-1] + (1 - self.decay_rates[0]) * grads
        v = self.decay_rates[1] * self.v_moment[-1] + (1 - self.decay_rates[1]) * grads**2
        
        self.m_moment = np.append(self.m_moment, m)
        self.v_moment = np.append(self.v_moment, v)
        
        corrected_m = m / (1 - self.decay_rates[0]**n_iter) 
        corrected_v = v / (1 - self.decay_rates[1]**n_iter)
        
        return corrected_m, corrected_v
    
    def train(self, X, y) -> None:
        pass
    
    def predict(self, X) -> np.ndarray:
        pass